# Import libraries

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from bokeh.palettes import Spectral5
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.palettes import Category20

from bokeh.io import export_png


output_notebook()

Loading BokehJS ...

--------------

The datasize is large. for this experimentation, i'll use only record for one month

# Read data for only april 2022

In [2]:
df = pd.read_parquet('data/yellow_tripdata_2022-04.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-04-01 00:21:13,2022-04-01 00:58:33,1.0,10.30,1.0,N,163,62,1,33.5,3.0,0.5,7.45,0.0,0.3,44.75,2.5,0.0
1,1,2022-04-01 00:07:47,2022-04-01 00:19:12,0.0,2.00,1.0,N,142,141,1,10.0,3.0,0.5,4.10,0.0,0.3,17.90,2.5,0.0
2,1,2022-04-01 00:14:52,2022-04-01 00:23:43,1.0,1.00,1.0,N,79,148,1,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,0.0
3,1,2022-04-01 00:30:02,2022-04-01 00:45:06,1.0,2.60,1.0,N,79,13,1,12.0,3.0,0.5,4.70,0.0,0.3,20.50,2.5,0.0
4,2,2022-04-01 00:48:40,2022-04-01 01:03:34,1.0,2.79,1.0,N,238,116,4,-12.5,-0.5,-0.5,0.00,0.0,-0.3,-13.80,0.0,0.0


In [4]:
df.isnull().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          118874
trip_distance                 0
RatecodeID               118874
store_and_fwd_flag       118874
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     118874
airport_fee              118874
dtype: int64

In [5]:
df.shape

(3599920, 19)

------------------

Large number of missing data. I'll filter row wise to reduce observations further

## Filter data

In [6]:
df = df[df['congestion_surcharge'].notnull()]

In [7]:
df.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
dtype: int64

In [8]:
df.shape

(3481046, 19)

----------------

No more missing values, but data size is still large.. I'll randomly select only 5000 rows

In [9]:
df = df[df['congestion_surcharge'].notnull()].sample(n=5000, random_state=42)

In [10]:
df.shape

(5000, 19)

## Assess data information

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 2617782 to 3156189
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               5000 non-null   int64         
 1   tpep_pickup_datetime   5000 non-null   datetime64[ns]
 2   tpep_dropoff_datetime  5000 non-null   datetime64[ns]
 3   passenger_count        5000 non-null   float64       
 4   trip_distance          5000 non-null   float64       
 5   RatecodeID             5000 non-null   float64       
 6   store_and_fwd_flag     5000 non-null   object        
 7   PULocationID           5000 non-null   int64         
 8   DOLocationID           5000 non-null   int64         
 9   payment_type           5000 non-null   int64         
 10  fare_amount            5000 non-null   float64       
 11  extra                  5000 non-null   float64       
 12  mta_tax                5000 non-null   float64       

In [12]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [13]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2617782,2,2022-04-23 16:38:04,2022-04-23 16:55:47,1.0,1.7,1.0,N,161,234,1,12.0,0.0,0.5,3.06,0.0,0.3,18.36,2.5,0.0
3419462,1,2022-04-30 15:19:30,2022-04-30 15:34:07,3.0,2.2,1.0,N,142,24,1,11.5,2.5,0.5,2.96,0.0,0.3,17.76,2.5,0.0


## Extract features

In [14]:
# extract the day of the week and hour of the day into separate columns
df['pickup_dayofweek'] = df['tpep_pickup_datetime'].dt.day_name()
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

df['dropoff_dayofweek'] = df['tpep_dropoff_datetime'].dt.day_name()
df['dropoff_hour'] = df['tpep_dropoff_datetime'].dt.hour

In [15]:
# # extract part of the day
# bins = [-1, 4, 11, 16, 23] # define the bins for different parts of day
# labels = ['late night', 'morning', 'afternoon', 'evening'] # define the labels for each bin

# df['pickup_partofday'] = pd.cut(df['pickup_hour'], bins=bins, labels=labels)
# df['dropoff_partofday'] = pd.cut(df['dropoff_hour'], bins=bins, labels=labels)

In [16]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_dayofweek,pickup_hour,dropoff_dayofweek,dropoff_hour
2617782,2,2022-04-23 16:38:04,2022-04-23 16:55:47,1.0,1.70,1.0,N,161,234,1,...,3.06,0.0,0.3,18.36,2.5,0.0,Saturday,16,Saturday,16
3419462,1,2022-04-30 15:19:30,2022-04-30 15:34:07,3.0,2.20,1.0,N,142,24,1,...,2.96,0.0,0.3,17.76,2.5,0.0,Saturday,15,Saturday,15
1654815,2,2022-04-15 08:34:48,2022-04-15 08:44:52,1.0,1.68,1.0,N,162,50,1,...,2.46,0.0,0.3,14.76,2.5,0.0,Friday,8,Friday,8
1247516,2,2022-04-11 17:35:06,2022-04-11 17:57:15,1.0,3.31,1.0,N,234,141,1,...,4.06,0.0,0.3,24.36,2.5,0.0,Monday,17,Monday,17
554511,2,2022-04-05 19:50:25,2022-04-05 19:54:40,1.0,0.62,1.0,N,161,170,2,...,0.00,0.0,0.3,8.80,2.5,0.0,Tuesday,19,Tuesday,19


In [17]:
df = df.reset_index(drop=True)

In [18]:
df['pickup_dayofweek'].unique()

array(['Saturday', 'Friday', 'Monday', 'Tuesday', 'Sunday', 'Thursday',
       'Wednesday'], dtype=object)

## Drop ID columns

In [19]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee',
       'pickup_dayofweek', 'pickup_hour', 'dropoff_dayofweek', 'dropoff_hour'],
      dtype='object')

In [20]:
df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'store_and_fwd_flag',
       'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee',
       'pickup_dayofweek', 'pickup_hour', 'dropoff_dayofweek', 'dropoff_hour']]

In [21]:
df.shape

(5000, 19)

# Data Visualization

In [22]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

## 1. Plot trip amount by distance

In [23]:
source = ColumnDataSource(df)
p = figure(title="Trip Distance vs. Fare Amount", x_axis_label="Trip Distance (miles)", y_axis_label="Fare Amount ($)", height=400)

# create a scatter plot of trip distance vs. fare amount
p.circle(df['trip_distance'], df['fare_amount'], size=2, color='navy')
show(p)

export_png(p, filename="img/fig1.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig1.png'

## 2. Plot hourly trip distance

In [24]:
# group the data by pickup hour and calculate total trip distance for each hour
hourly_distance = df.groupby(df.pickup_hour)['trip_distance'].sum().reset_index()

# create a ColumnDataSource object to use as the data source for the plot
source = ColumnDataSource(hourly_distance)

# create a figure with a title and axis labels
p = figure(title="Total Trip Distance by Pickup Hour", x_axis_label="Pickup Hour", 
           y_axis_label="Total Trip Distance", height =400)

# add a bar chart to the figure
p.vbar(x='pickup_hour', top='trip_distance', source=source, width=0.8, color=Category20[20][0])

# show the plot
show(p)

export_png(p, filename="img/fig2.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig2.png'

## 3. Plot pickup hour by number of trips

In [25]:
grouped = df.groupby('pickup_hour').size().reset_index(name='num_trips')
source = ColumnDataSource(grouped)
p = figure(title='Pickup hour vs. Number of trips', x_axis_label='Pickup hour', y_axis_label='Number of trips', height=400)
p.line(x='pickup_hour', y='num_trips', source=source)
show(p)

export_png(p, filename="img/fig3.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig3.png'

## 4. Compute mean congestion_surcharge for each dropoff_dayofweek

In [26]:
mean_congestion_surcharge = df.groupby('dropoff_dayofweek')['congestion_surcharge'].mean()

# create a ColumnDataSource object
source = ColumnDataSource(data=dict(
    dropoff_dayofweek = mean_congestion_surcharge.index.values,
    mean_congestion_surcharge = mean_congestion_surcharge.values
))

# create the figure
p = figure(x_range=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], height=350, title='Average Congestion Surcharge by Dropoff Day of Week')

# create the bar chart
p.vbar(x='dropoff_dayofweek', top='mean_congestion_surcharge', source=source, width=0.5,
       fill_color=factor_cmap('dropoff_dayofweek', palette=Spectral6, factors=mean_congestion_surcharge.index.values))

# set the axis labels
p.xaxis.axis_label = 'Dropoff Day of Week'
p.yaxis.axis_label = 'Mean Congestion Surcharge'

# show the plot
show(p)

export_png(p, filename="img/fig4.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig4.png'

## 5. Fare Amount vs Airport Fee

In [27]:
grouped = df.groupby('fare_amount').size().reset_index(name='airport_fee')
source = ColumnDataSource(grouped)
p = figure(title='Fare Amout  vs. Airport Fee', x_axis_label='Fare Amout', y_axis_label='Airport Fee', height=400)
p.line(x='fare_amount', y='airport_fee', source=source)
show(p)

export_png(p, filename="img/fig5.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig5.png'

## 6. Plot histogram of selected features as subplot

In [28]:
# create histograms to show distribution
p1 = figure(title="Passenger Count Distribution", x_axis_label="Passenger Count", y_axis_label="Frequency", height=300, width=500)
hist, edges = np.histogram(df['passenger_count'], bins=range(10))
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='navy', line_color='white')

# create a figure with a title and axis labels
p2 = figure(title="Distance Distribution", x_axis_label="Trip Distance", y_axis_label="Frequency", height=300, width=500)
hist, edges = np.histogram(df['trip_distance'], bins=50)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='navy', line_color='white')

# create a figure with a title and axis labels
p3 = figure(title="Payment Type Distribution", x_axis_label="Payment Type", y_axis_label="Frequency", height=300, width=500)
hist, edges = np.histogram(df['payment_type'])
p3.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='navy', line_color='white')

# create a figure with a title and axis labels
p4 = figure(title="Amount Distribution", x_axis_label="Total Amount", y_axis_label="Frequency", height=300, width=500)
hist, edges = np.histogram(df['total_amount'], bins=60)
p4.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='navy', line_color='white')

# create a grid of plots and show them
grid = gridplot([[p1, p2], [p3, p4]])
show(grid)

export_png(grid, filename="img/fig6.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig6.png'

## 7. Plot Payment type vs. Total amount

In [29]:
grouped = df.groupby('payment_type').agg({'total_amount': 'mean'})
grouped = grouped.reset_index()
source = ColumnDataSource(grouped)
p5 = figure(title='Payment type vs. Total amount', x_axis_label='Payment type', y_axis_label='Total amount', height=300, width=500)
p5.vbar(x='payment_type', top='total_amount', width=0.5, source=source)

# Plot 6 - Congestion surcharge vs. Total amount
grouped = df.groupby('congestion_surcharge').agg({'total_amount': 'mean'})
grouped = grouped.reset_index()
source = ColumnDataSource(grouped)
p6 = figure(title='Congestion surcharge vs. Total amount', x_axis_label='Congestion surcharge', y_axis_label='Total amount', height=300, width=500)
p6.vbar(x='congestion_surcharge', top='total_amount', width=0.5, source=source)

# Create a grid of plots and show them
grid = gridplot([[p5, p6]])
show(grid)

export_png(grid, filename="img/fig7.png")

'C:\\Users\\Chizzy\\Documents\\Notebooks\\Bokeh\\img\\fig7.png'

## Types of plots available

-------------

Line plot: `p.line`

Scatter plot: `p.scatter` or `p.circle`

Bar plot: `p.vbar` or `p.hbar`

Stacked bar plot: `p.vbar_stack` or `p.hbar_stack`

Area plot: `p.varea` or `p.harea`

Stacked area plot: `p.varea_stack` or `p.harea_stack`

Heatmap plot: `p.rect` or `p.image`

Box plot: `p.boxplot`

Violin plot: `p.violin`

Donut plot: `p.donut`

Pie chart: `p.wedge`

Polar plot: `p.polar`